In [233]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

import sympy as sp
import numpy as np

In [234]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init, order, r = get_processed_and_shifted_recurrence(laplace2d)

In [235]:
r

(-1)**(n + 1)*((-1)**(n - 3)*(n + (n - 2)**3 - 2*(n - 2)**2 - 2)*s(n - 3)/(x0**3 + x0*x1**2) + (-1)**(n - 2)*(-n + 3*(n - 2)**2 + 2)*s(n - 2)/(x0**2 + x1**2) + (-1)**(n - 1)*(3*x0**2*(n - 2) + x0**2 + x1**2*(n - 2) - x1**2)*s(n - 1)/(x0**3 + x0*x1**2))

In [285]:
#We want to subsitute s(i) r^i_{ct} = g(i)
g = sp.Function("g")
s = sp.Function("s")
n = sp.symbols("n")
rct = sp.symbols("r_{ct}")

In [286]:
r_new = r*rct**n
for i in range(order):
    r_new = r_new.subs(s(n-i),g(n-i)/(rct**(n-i)))

In [287]:
r_new

(-1)**(n + 1)*r_{ct}**n*((-1)**(n - 3)*r_{ct}**(3 - n)*(n + (n - 2)**3 - 2*(n - 2)**2 - 2)*g(n - 3)/(x0**3 + x0*x1**2) + (-1)**(n - 2)*r_{ct}**(2 - n)*(-n + 3*(n - 2)**2 + 2)*g(n - 2)/(x0**2 + x1**2) + (-1)**(n - 1)*r_{ct}**(1 - n)*(3*x0**2*(n - 2) + x0**2 + x1**2*(n - 2) - x1**2)*g(n - 1)/(x0**3 + x0*x1**2))

In [303]:
var = _make_sympy_vec("x", 2)
var_t = _make_sympy_vec("t", 2)
g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
derivs = [sp.diff(g_x_y,
                    var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                    for i in range(15)]
max_abs = .0000001
x_coord = np.random.rand()*max_abs  # noqa: NPY002
y_coord = np.random.rand()*max_abs
coord_dict = {var[0]: x_coord, var[1]: y_coord}

In [304]:
def evaluate_recurrence(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    for i in range(2, p):
        subs_dict[g(i)] = recur.subs(n, i).subs(subs_dict).subs(coord_dict).subs(rct, rct_val)
    return np.array(list(subs_dict.values()))

In [305]:
def evaluate_true(coord_dict, rct_val, p):
    retMe = []
    for i in range(p):
        retMe.append(derivs[i].subs(coord_dict).subs(rct, rct_val)*rct_val**i)
    return np.array(retMe)

In [ ]:
exp = evaluate_recurrence(coord_dict, np.sqrt(x_coord**2 + y_coord**2), r_new, 14) 
true = evaluate_true(coord_dict, np.sqrt(x_coord**2 + y_coord**2), 14)
np.abs(exp-true)/np.abs(true)

array([0, 0, 2.51782700048972e-16, 4.95906080244864e-16,
       3.99749493419410e-16, 1.25768654288579e-15, 1.72518029710139e-14,
       4.46968552937884e-15, 3.38121217373531e-14, 5.13481400668422e-14],
      dtype=object)